## Персептрон (253)

In [26]:
from theory.linear_algebra import Vector, dot

# Ступенчатая функция
def step_function(x):
    return 1 if x >= 0 else 0

def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    """Возвращает 1, если персептрон 'активируется', и 0, если нет"""
    calculation = dot(weights, x) + bias
    return step_function(calculation)

### Логический вентиль И

In [27]:
and_weights = [2., 2]
and_bias = -3

assert perceptron_output(and_weights, and_bias, [1, 1]) == 1
assert perceptron_output(and_weights, and_bias, [0, 1]) == 0
assert perceptron_output(and_weights, and_bias, [1, 0]) == 0
assert perceptron_output(and_weights, and_bias, [0, 0]) == 0

### Логический вентиль ИЛИ

In [28]:
or_weights = [2., 2]
or_bias = -1

assert perceptron_output(or_weights, or_bias, [1, 1]) == 1
assert perceptron_output(or_weights, or_bias, [0, 1]) == 1
assert perceptron_output(or_weights, or_bias, [1, 0]) == 1
assert perceptron_output(or_weights, or_bias, [0, 0]) == 0

### Логический вентиль НЕ

In [29]:
not_weights = [-2.]
not_bias = 1

assert perceptron_output(not_weights, not_bias, [0]) == 1
assert perceptron_output(not_weights, not_bias, [1]) == 0

## Нейронные сети прямого распространения

### Сигмоидальная функция (step_function не подойдет, т.к. нам нужно дифф исчисление => нужна гладкая ф-я )

In [30]:
import math 

def sigmoid(t: float) -> float:
    return 1 / (1 + math.exp(-t))

### Вычисляем выход

#### представим нейронную сеть как список (слоев) списков (нейронов) векторов(весов)

In [31]:
def neuron_output(weights: Vector, inputs: Vector) -> float:
    # weights включает член смещения, inputs включает единицу
    return sigmoid(dot(weights,inputs))

### Используем нейронную сеть

In [32]:
from typing import List

def feed_forward(neural_network: List[List[Vector]],
                 input_vector: Vector) -> List[Vector]:
    """Пропускает входной вектор через нейронную сеть.
       Возвращает все слои (а не только последний)"""
    outputs: List[Vector] = []

    for layer in neural_network:
        input_with_bias = input_vector + [1] # добавляем константу
        output = [neuron_output(neuron, input_with_bias)    # Вычислить выход
                  for neuron in layer]                      # для каждого нейрона.
        outputs.append(output)                              # Сложить результаты

        # Затем вход в следующий слой является выходом этого слоя
        input_vector = output

    return outputs

### Логический вентиль исключающий ИЛИ (XOR)

In [33]:
xor_network = [# Скрытый слой
              [[20., 20, -30],      # Нейрон 'И'
               [20., 20, -10]],     # Нейрон 'ИЛИ'
              # Выходной слой
              [[-60., 60, -30]]]    # Нейрон '2-й вход, но не 1-й вход'

# Функция feed_forward возвращает входы всех слоев, поэтому
# [-1] получает окончательный выход и 
# [0] получает значение из результирующего вектора
assert 0.000 < feed_forward(xor_network, [0, 0])[-1][0] < 0.001
assert 0.999 < feed_forward(xor_network, [1, 0])[-1][0] < 1.000
assert 0.999 < feed_forward(xor_network, [0, 1])[-1][0] < 1.001
assert 0.000 < feed_forward(xor_network, [1, 1])[-1][0] < 0.001

## Обратное распространение

### Ф-ия для вычисления градиентов

In [34]:
def sqerror_gradients(network: List[List[Vector]],
                      input_vector: Vector,
                      target_vector: Vector) -> List[List[Vector]]:
    """С учетом нейронной сети, вектора входов и вектора целей 
       сделать предсказание и вычислить градиент потери, т.е.
       сумму квадратов ошибок по отношению к весам нейрона"""
    # Прямое прохождение 
    hidden_outputs, outputs = feed_forward(network, input_vector)

    # Градиенты по отношению к преактивационным выходам выходного нейрона
    output_deltas = [output * (1 - output) * (output - target)
                     for output, target in zip(outputs, target_vector)]
    
    # Градиенты по отношению к весам выходного нейрона
    output_grads = [[output_deltas[i] * hidden_output
                     for hidden_output in hidden_outputs + [1]]
                    for i, output_neuron in enumerate(network[-1])]

    # Градиенты по отношению к преактивационным выходам скрытого нейрона
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                         dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # Градиенты по отношению к весам скрытого нейрона
    hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                    for i, hidden_neuron in enumerate(network[0])]

    return [hidden_grads, output_grads]

### Генерируем тренировочные данные и инициализирем нашу нейронную сеть случайными весами

In [35]:
import random
random.seed(0)
    
# Тренировочные данные
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]]
    
# Начать со случайных весов
network = [ # Скрытый слой: 2 выхода -> 2 выхода
            [[random.random() for _ in range(2 + 1)],   # 1-й скрытый слой
             [random.random() for _ in range(2 + 1)]],  # 2-й скрытый слой
            # Выходной слой: 2 входа -> 1 выход
            [[random.random() for _ in range(2 + 1)]]   # 1-й выхю нейрон
            ]

### Тренируем с помощью град спуска

#### т.к у нас есть несколько векторов параметров, каждый со своим градиентом, а значит, нам придется вызвать ф-ю gradient_step для каждого из них

In [36]:
from gradient_descent import gradient_step
import tqdm

learning_rate = 1.0

for epoch in tqdm.trange(20000, desc="neural net for xor"):
        for x, y in zip(xs, ys):
            gradients = sqerror_gradients(network, x, y)
    
            # Сделать градиентный шаг для каждого нейрона в каждом слое
            network = [[gradient_step(neuron, grad, -learning_rate)
                        for neuron, grad in zip(layer, layer_grad)]
                       for layer, layer_grad in zip(network, gradients)]
    
# Проверить, что сеть усвоила XOR
assert feed_forward(network, [0, 0])[-1][0] < 0.01
assert feed_forward(network, [0, 1])[-1][0] > 0.99
assert feed_forward(network, [1, 0])[-1][0] > 0.99
assert feed_forward(network, [1, 1])[-1][0] < 0.01

neural net for xor: 100%|██████████| 20000/20000 [00:00<00:00, 23030.33it/s]


## Задача Fizz Buzz

### Функция для генерирования векторов целей

In [37]:
def fizz_buzz_encode(x: int) -> Vector:
    if x % 15 == 0:
        return [0, 0, 0, 1]
    elif x % 5 == 0:
        return [0, 0, 1, 0]
    elif x % 3 == 0:
        return [0, 1, 0, 0]
    else:
        return [1, 0, 0, 0]
    
assert fizz_buzz_encode(2) == [1, 0, 0, 0]
assert fizz_buzz_encode(6) == [0, 1, 0, 0]
assert fizz_buzz_encode(10) == [0, 0, 1, 0]
assert fizz_buzz_encode(30) == [0, 0, 0, 1]

### Конвертируем числа в двоичные представления

In [38]:
def binary_encode(x: int) -> Vector:
    binary: List[float] = []

    for i in range(10):
        binary.append(x % 2)
        x = x // 2

    return binary

### Тренировочный набор

In [39]:
xs = [binary_encode(n) for n in range(101, 1024)]
ys = [fizz_buzz_encode(n) for n in range(101, 1024)]

### Создадим нейронную сеть

In [42]:
NUM_HIDDEN = 25

network = [
    # Скрытый слой: 10 входов -> NUM_HIDDEN выходов
    [[random.random() for _ in range(10 + 1)] for _ in range(NUM_HIDDEN)],

    # Выходной слой: NUM_HIDDEN входов -> 4 выхода
    [[random.random() for _ in range(NUM_HIDDEN + 1)] for _ in range(4)]
]

### Тренируем

In [43]:
from theory.linear_algebra import squared_distance

learning_rate = 1.0

with tqdm.trange(500) as t:
        for epoch in t:
            epoch_loss = 0.0
    
            for x, y in zip(xs, ys):
                predicted = feed_forward(network, x)[-1]
                epoch_loss += squared_distance(predicted, y)
                gradients = sqerror_gradients(network, x, y)
    
                # Сделать градиентный шаг для каждого нейрона в каждом слое
                network = [[gradient_step(neuron, grad, -learning_rate)
                            for neuron, grad in zip(layer, layer_grad)]
                        for layer, layer_grad in zip(network, gradients)]
    
            t.set_description(f"fizz buzz (loss: {epoch_loss:.2f})")

fizz buzz (loss: 16.22): 100%|██████████| 500/500 [01:32<00:00,  5.38it/s] 


### Наша сеть будет производить 4-мерный вектор, но нужно одно предсказание

In [44]:
def argmax(xs: list) -> int:
    """Возвращает индекс наибольшего значения"""
    return max(range(len(xs)), key=lambda i: xs[i])

assert argmax([0, -1]) == 0
assert argmax([-1, 0]) == 1
assert argmax([-1, 10, 5, 20, -3]) == 3

### Решение задачи

In [45]:
num_correct = 0
    
for n in range(1, 101):
    x = binary_encode(n)
    predicted = argmax(feed_forward(network, x)[-1])
    actual = argmax(fizz_buzz_encode(n))
    labels = [str(n), "fizz", "buzz", "fizzbuzz"]
    print(n, labels[predicted], labels[actual])
    
    if predicted == actual:
        num_correct += 1
    
print(num_correct, "/", 100)

1 1 1
2 2 2
3 fizz fizz
4 4 4
5 5 buzz
6 fizz fizz
7 7 7
8 8 8
9 fizz fizz
10 buzz buzz
11 11 11
12 fizz fizz
13 13 13
14 14 14
15 fizzbuzz fizzbuzz
16 16 16
17 17 17
18 fizz fizz
19 19 19
20 20 buzz
21 fizz fizz
22 22 22
23 23 23
24 fizz fizz
25 buzz buzz
26 26 26
27 fizz fizz
28 28 28
29 29 29
30 fizzbuzz fizzbuzz
31 31 31
32 32 32
33 fizz fizz
34 34 34
35 buzz buzz
36 fizz fizz
37 37 37
38 38 38
39 fizz fizz
40 buzz buzz
41 41 41
42 fizz fizz
43 43 43
44 44 44
45 fizzbuzz fizzbuzz
46 46 46
47 47 47
48 fizz fizz
49 49 49
50 buzz buzz
51 fizz fizz
52 52 52
53 53 53
54 fizz fizz
55 buzz buzz
56 56 56
57 fizz fizz
58 58 58
59 59 59
60 fizzbuzz fizzbuzz
61 61 61
62 62 62
63 fizz fizz
64 64 64
65 65 buzz
66 fizz fizz
67 67 67
68 68 68
69 fizz fizz
70 buzz buzz
71 71 71
72 fizz fizz
73 73 73
74 74 74
75 fizzbuzz fizzbuzz
76 76 76
77 77 77
78 fizz fizz
79 79 79
80 80 buzz
81 fizz fizz
82 82 82
83 83 83
84 fizz fizz
85 buzz buzz
86 86 86
87 fizz fizz
88 88 88
89 89 89
90 fizzbuzz fizzbuzz
91